# PCA notes

In [2]:
import numpy as np
import pandas as pd
import visuals as vs
from IPython.display import display # 使得我们可以对DataFrame使用display()函数

# 设置以内联的形式显示matplotlib绘制的图片（在notebook中显示更美观）
%matplotlib inline

# 载入整个客户数据集
try:
    data = pd.read_csv("customers.csv")
    data.drop(['Region', 'Channel'], axis = 1, inplace = True)
except:
    print ("Dataset could not be loaded. Is the dataset missing?")

In [28]:
# 特征缩放
log_data = np.log(data)

# 清理异常值： 定义为多于一个维度中， 位于 [Q1 - 1.5IQR, Q3 + 1.5IQR] 范围外的点
outliers  = []
for feature in log_data.keys():
    Q1 = np.percentile(log_data[feature], 25)
    Q3 = np.percentile(log_data[feature], 75)
    IQR = Q3 - Q1
    outlier_data = log_data[~((log_data[feature] >= Q1 - 1.5 * IQR) & (log_data[feature] <= Q3 + 1.5 * IQR))]
    outliers.extend(outlier_data.index.values)

from collections import Counter
counter = Counter(outliers)
# 多于一个特征下被看作是异常的数据点
outliers2 = [k for k in counter.keys() if counter[k] > 1]
# display(data.iloc[outliers2])

good_data = log_data.drop(log_data.index[outliers2]).reset_index(drop = True)
print(good_data.shape)

(435, 6)


## PCA (mannually)


In [45]:
# 转成np matrix
test_data = good_data.as_matrix().transpose()   # D x N

def normalize_data(d):
    n, m = d.shape
    mean = np.mean(d, axis=1, keepdims=True)
    var = np.var(d, axis=1, keepdims=True)
    return (d - mean) / np.sqrt(var)

# test_data = normalize_data(test_data)
print(test_data.shape)

# set numpy print precision，Small results can be suppressed
np.set_printoptions(precision=4, suppress=True)  # default: percision=8, suppress=False

(6, 435)


In [46]:
# PCA分析
# 1.计算样本协方差矩阵
n, m = test_data.shape

def covariance_(a, b):
    n, m = a.shape
    assert((n, m) == b.shape)
    mean_a = np.mean(a, axis=1, keepdims=True)
    mean_b = np.mean(b, axis=1, keepdims=True)
    return np.dot(a - mean_a, (b - mean_b).transpose()) / (m - 1)

def variance_(d):
    n, m = d.shape
    mean = np.mean(d, axis=1, keepdims=True)
    # print(mean)
    return np.dot(d - mean, (d - mean).transpose()) / (m - 1)

def correlation_(a, b):
    cov = covariance_(a, b)
    var_a = variance_(a)
    var_b = variance_(b)
    n = cov.shape[0]
    cor = np.zeros(cov.shape)
    for i in range(n):
        for j in range(n):
            cor[i][j] = cov[i][j] / np.sqrt(var_a[i][i] * var_b[j][j])
    return cor

covariance = np.cov(test_data)   # D x D
print(covariance)
correlation = np.corrcoef(test_data)  # D x D
print(correlation)


[[ 2.0253 -0.0365 -0.1929  0.6295 -0.3572  0.3814]
 [-0.0365  1.1298  0.8662 -0.0864  1.2041  0.445 ]
 [-0.1929  0.8662  1.0943 -0.2239  1.3929  0.3128]
 [ 0.6295 -0.0864 -0.2239  1.5902 -0.4564  0.3595]
 [-0.3572  1.2041  1.3929 -0.4564  2.8377  0.3402]
 [ 0.3814  0.445   0.3128  0.3595  0.3402  1.5903]]
[[ 1.     -0.0241 -0.1296  0.3507 -0.149   0.2125]
 [-0.0241  1.      0.779  -0.0645  0.6725  0.332 ]
 [-0.1296  0.779   1.     -0.1697  0.7904  0.2371]
 [ 0.3507 -0.0645 -0.1697  1.     -0.2148  0.2261]
 [-0.149   0.6725  0.7904 -0.2148  1.      0.1601]
 [ 0.2125  0.332   0.2371  0.2261  0.1601  1.    ]]


In [18]:
# 2.计算 covariance矩阵 特征值，特征向量，排序，做一些验证
eigenvalues, eigenvectors = np.linalg.eig(covariance)
sorted_indices = np.argsort(eigenvalues)[::-1]   # 降序排序

eigenvalues = eigenvalues[sorted_indices]
eigenvectors = eigenvectors[:, sorted_indices]
print('eigenvalues:\n', eigenvalues)
print('eigenvectors:\n', eigenvectors)

print('Av:\n', np.dot(covariance, eigenvectors))
print('lv:\n', eigenvalues * eigenvectors)

print('v[0]:\n', eigenvectors[:, 0])    # v 是列向量
print('Av[0]:\n', np.dot(covariance, eigenvectors[:, 0]))


eigenvalues:
 [ 4.5488  2.7085  1.2636  1.0392  0.498   0.2095]
eigenvectors:
 [[-0.1675  0.6859  0.6774  0.2043 -0.0026  0.0292]
 [ 0.4014  0.1672 -0.0402 -0.0128  0.7192 -0.5402]
 [ 0.4381  0.0707  0.0195 -0.0557  0.3554  0.8205]
 [-0.1782  0.5005 -0.315  -0.7854 -0.0331  0.0205]
 [ 0.7514  0.0424  0.2117 -0.2096 -0.5582 -0.1824]
 [ 0.1499  0.4941 -0.6286  0.5423 -0.2092  0.0197]]
Av:
 [[-0.7618  1.8579  0.8559  0.2123 -0.0013  0.0061]
 [ 1.8261  0.4529 -0.0508 -0.0133  0.3582 -0.1132]
 [ 1.9929  0.1916  0.0247 -0.0579  0.177   0.1719]
 [-0.8106  1.3557 -0.3981 -0.8162 -0.0165  0.0043]
 [ 3.4181  0.115   0.2675 -0.2178 -0.278  -0.0382]
 [ 0.6817  1.3384 -0.7942  0.5636 -0.1042  0.0041]]
lv:
 [[-0.7618  1.8579  0.8559  0.2123 -0.0013  0.0061]
 [ 1.8261  0.4529 -0.0508 -0.0133  0.3582 -0.1132]
 [ 1.9929  0.1916  0.0247 -0.0579  0.177   0.1719]
 [-0.8106  1.3557 -0.3981 -0.8162 -0.0165  0.0043]
 [ 3.4181  0.115   0.2675 -0.2178 -0.278  -0.0382]
 [ 0.6817  1.3384 -0.7942  0.5636 -0.1042 

$Av = lv$，  $l$是特征值，$v$是特征向量

下面验证： $A = E^T \Lambda E$

In [47]:
print('covariance:\n', covariance)
print('digonal matrix:\n', eigenvalues * np.identity(n))
print('decomposition:\n', np.dot(np.dot(eigenvectors, eigenvalues * np.identity(n)), eigenvectors.transpose()))

covariance:
 [[ 2.0253 -0.0365 -0.1929  0.6295 -0.3572  0.3814]
 [-0.0365  1.1298  0.8662 -0.0864  1.2041  0.445 ]
 [-0.1929  0.8662  1.0943 -0.2239  1.3929  0.3128]
 [ 0.6295 -0.0864 -0.2239  1.5902 -0.4564  0.3595]
 [-0.3572  1.2041  1.3929 -0.4564  2.8377  0.3402]
 [ 0.3814  0.445   0.3128  0.3595  0.3402  1.5903]]
digonal matrix:
 [[ 4.5488  0.      0.      0.      0.      0.    ]
 [ 0.      2.7085  0.      0.      0.      0.    ]
 [ 0.      0.      1.2636  0.      0.      0.    ]
 [ 0.      0.      0.      1.0392  0.      0.    ]
 [ 0.      0.      0.      0.      0.498   0.    ]
 [ 0.      0.      0.      0.      0.      0.2095]]
decomposition:
 [[ 2.0253 -0.0365 -0.1929  0.6295 -0.3572  0.3814]
 [-0.0365  1.1298  0.8662 -0.0864  1.2041  0.445 ]
 [-0.1929  0.8662  1.0943 -0.2239  1.3929  0.3128]
 [ 0.6295 -0.0864 -0.2239  1.5902 -0.4564  0.3595]
 [-0.3572  1.2041  1.3929 -0.4564  2.8377  0.3402]
 [ 0.3814  0.445   0.3128  0.3595  0.3402  1.5903]]


In [48]:
# 3.计算转换的数据，计算协方差矩阵，应当是特征值对角矩阵

transform_data = np.dot(eigenvectors.transpose(), test_data)
print(transform_data.shape)
print('covariance matrix of transform_data:\n', np.cov(transform_data))


(6, 435)
covariance matrix of transform_data:
 [[ 4.5488 -0.      0.      0.     -0.      0.    ]
 [-0.      2.7085  0.      0.      0.     -0.    ]
 [ 0.      0.      1.2636 -0.     -0.     -0.    ]
 [ 0.      0.     -0.      1.0392 -0.      0.    ]
 [-0.      0.     -0.     -0.      0.498  -0.    ]
 [ 0.     -0.     -0.      0.     -0.      0.2095]]


$ND = E^T D$

In [49]:

# 4.计算转换后的数据与原始数据变量之间的相关系数 (PCA讲义上用这个分析相关性，但是感觉没什么用)

print('eigenvectors(PC transform matrix):\n', eigenvectors)   # e1: eigenvectors[:, 0], 第一列的
cov_yx = covariance_(transform_data, test_data)

print('covariance of PC and origin variables:\n', covariance_(transform_data, test_data).transpose())
# print('calc: covariance of components and origin variables:\n', np.dot(eigenvectors, eigenvalues * np.identity(n)))
print('correlation of PC and origin variables:\n', correlation_(transform_data, test_data).transpose())
# print('correlation of components and origin variables:\n', np.corrcoef(transform_data, test_data).transpose())
print('correlation of origin variables:\n', correlation_(test_data, test_data).transpose())

# print('calc:\n', np.dot(eigenvalues * np.identity(n), eigenvectors.transpose()))

## normalize 之后再计算一遍， PCA 不用correlation， 而是用covariance， 为什么？


eigenvectors(PC transform matrix):
 [[-0.1675  0.6859  0.6774  0.2043 -0.0026  0.0292]
 [ 0.4014  0.1672 -0.0402 -0.0128  0.7192 -0.5402]
 [ 0.4381  0.0707  0.0195 -0.0557  0.3554  0.8205]
 [-0.1782  0.5005 -0.315  -0.7854 -0.0331  0.0205]
 [ 0.7514  0.0424  0.2117 -0.2096 -0.5582 -0.1824]
 [ 0.1499  0.4941 -0.6286  0.5423 -0.2092  0.0197]]
covariance of PC and origin variables:
 [[-0.7618  1.8579  0.8559  0.2123 -0.0013  0.0061]
 [ 1.8261  0.4529 -0.0508 -0.0133  0.3582 -0.1132]
 [ 1.9929  0.1916  0.0247 -0.0579  0.177   0.1719]
 [-0.8106  1.3557 -0.3981 -0.8162 -0.0165  0.0043]
 [ 3.4181  0.115   0.2675 -0.2178 -0.278  -0.0382]
 [ 0.6817  1.3384 -0.7942  0.5636 -0.1042  0.0041]]
correlation of PC and origin variables:
 [[-0.251   0.7932  0.535   0.1463 -0.0013  0.0094]
 [ 0.8055  0.2589 -0.0425 -0.0123  0.4775 -0.2326]
 [ 0.8932  0.1113  0.021  -0.0543  0.2397  0.3591]
 [-0.3014  0.6533 -0.2808 -0.6349 -0.0185  0.0075]
 [ 0.9514  0.0415  0.1413 -0.1268 -0.2338 -0.0496]
 [ 0.2535  0.6

[PCA讲义](https://onlinecourses.science.psu.edu/stat505/node/54) 分析了PC和原变量之间的相关系数， 其实可以看出来，第一个PC和 X2、X3、X5 即Milk、Grocery、Detergents_Paper 的相关系数都超过0.8。转换系数（即e1）分别是0.4， 0.43、0.75。 

这个说明 PC1 具有同时购买X2、X3、X5的行为。 PC1 并不能表示某种类型的客户， 但是可以表示某种内在的行为联系： 如果购买X2，有很大可能同时购买X3、X5。

In [40]:

# 5.与sklearn的PCA比较
from sklearn.decomposition import PCA
pca = PCA(n_components=6).fit(good_data)

# # TODO：使用上面的PCA拟合将变换施加在log_samples上
# pca_samples = pca.transform(log_samples)

print('pca.explained_variance_:\n', pca.explained_variance_)
print('eigenvalues:\n', eigenvalues)

print('pca.components_:\n', pca.components_.transpose())
print('eigenvectors:\n', eigenvectors)

pca.explained_variance_:
 [ 4.5384  2.7023  1.2607  1.0368  0.4969  0.2091]
eigenvalues:
 [ 4.5488  2.7085  1.2636  1.0392  0.498   0.2095]
pca.components_:
 [[ 0.1675 -0.6859 -0.6774 -0.2043 -0.0026  0.0292]
 [-0.4014 -0.1672  0.0402  0.0128  0.7192 -0.5402]
 [-0.4381 -0.0707 -0.0195  0.0557  0.3554  0.8205]
 [ 0.1782 -0.5005  0.315   0.7854 -0.0331  0.0205]
 [-0.7514 -0.0424 -0.2117  0.2096 -0.5582 -0.1824]
 [-0.1499 -0.4941  0.6286 -0.5423 -0.2092  0.0197]]
eigenvectors:
 [[-0.1675  0.6859  0.6774  0.2043 -0.0026  0.0292]
 [ 0.4014  0.1672 -0.0402 -0.0128  0.7192 -0.5402]
 [ 0.4381  0.0707  0.0195 -0.0557  0.3554  0.8205]
 [-0.1782  0.5005 -0.315  -0.7854 -0.0331  0.0205]
 [ 0.7514  0.0424  0.2117 -0.2096 -0.5582 -0.1824]
 [ 0.1499  0.4941 -0.6286  0.5423 -0.2092  0.0197]]


数值居然对不上。。。。。
eigentvectors和 pca大致相同， 符号除了最后一个，其他都是相反的。 特征向量符号相反不影响。

In [41]:
pca.transform(good_data)

array([[-1.758 ,  0.0097, -0.959 , -1.6824,  0.268 , -0.3891],
       [-1.7887, -0.8123,  0.2315, -0.0036,  0.1194, -0.2106],
       [-1.8834, -1.5991,  1.3204, -0.5432, -0.3934, -0.3117],
       ..., 
       [-3.7425, -0.8561, -0.9885, -0.8879,  0.0503,  0.2058],
       [ 1.6691, -0.398 ,  0.5161, -1.3189,  0.0913,  0.0056],
       [ 0.739 ,  3.6914, -2.0335, -0.9927,  0.3109, -0.1734]])

In [43]:
np.dot(eigenvectors.transpose(), test_data - np.mean(test_data, axis=1, keepdims=True)).transpose()

array([[ 1.758 , -0.0097,  0.959 ,  1.6824,  0.268 , -0.3891],
       [ 1.7887,  0.8123, -0.2315,  0.0036,  0.1194, -0.2106],
       [ 1.8834,  1.5991, -1.3204,  0.5432, -0.3934, -0.3117],
       ..., 
       [ 3.7425,  0.8561,  0.9885,  0.8879,  0.0503,  0.2058],
       [-1.6691,  0.398 , -0.5161,  1.3189,  0.0913,  0.0056],
       [-0.739 , -3.6914,  2.0335,  0.9927,  0.3109, -0.1734]])

在做PCA转换时， 将坐标中心位置做了一次平移。 转换后的数据的均值为坐标原点

还是有符号差异。。。